In [50]:
import pandas as pd
from capping import capping
from coinmarketcap import Market
from bittrex import Bittrex, API_V2_0, API_V1_1
from os import getenv

# get coinmarketcap top 50
coinmarketcap = Market()
data = coinmarketcap.ticker(limit=50)

# create blacklist
blacklist = pd.Series(['BCC','USDT'])
symbol_map = {'IOTA': 'MIOTA',
              'BCC': 'BCH'}


## binance
from binance.client import Client
my_binance = Client(getenv('BINANCE_API_KEY'), getenv('BINANCE_API_SECRET'))

# binance markets
binance_markets = my_binance.get_products()

binance_coins = pd.Series(list(set([p['baseAsset'] for p in binance_markets['data']])))
binance_coins = binance_coins.replace(symbol_map)

# create dataframe from coinmarketcap data
df = pd.DataFrame(data)

# remove blacklist coins and coins not in bittrex from dataframe
df = df.loc[df['symbol'].isin(binance_coins) & ~df['symbol'].isin(blacklist), :]


# get top 20
df = df.head(17)

# compute market weights
df['market_cap_usd'] = df['market_cap_usd'].astype(float)
df['weight'] = df['market_cap_usd']/df['market_cap_usd'].sum()

# compute capped weights
capped = capping(df, 0.1, weight_column='weight')

weights = capped.loc[:, ['name', 'symbol', 'weight', 'price_btc']]
weights['price_btc'] = weights['price_btc'].astype(float)
weights.set_index('symbol', inplace=True)
weights

,name,weight,price_btc
symbol,,,
BTC,Bitcoin,0.100000,1.000000
ETH,Ethereum,0.100000,0.052953
BCH,Bitcoin Cash,0.100000,0.209343
XRP,Ripple,0.100596,0.000075
LTC,Litecoin,0.100596,0.019871
MIOTA,IOTA,0.088965,0.000328
ADA,Cardano,0.075867,0.000030
DASH,Dash,0.068089,0.089719
BTG,Bitcoin Gold,0.042118,0.025781


In [51]:
my_account = pd.DataFrame([b for b in my_binance.get_account()['balances'] if float(b['free']) > 0])
my_account['asset'] = my_account['asset'].replace(symbol_map)
my_account.set_index('asset', inplace=True)

In [52]:
weights['current_value'] = my_account['free'].astype(float)
weights['current_value_btc'] = weights['current_value'] * weights['price_btc'].astype('float')
weights.fillna(0, inplace=True)
weights['target_value_btc'] = weights['weight'] * weights['current_value_btc'].sum()
weights['diff_btc'] = weights['target_value_btc'] - weights['current_value_btc']
weights['diff_btc_round'] = weights['diff_btc'].round(4)
weights['diff'] = weights['diff_btc_round']/weights['price_btc']

In [53]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if val < 0:
        color = 'red' 
    elif val > 0:
        color = 'green'
    else:
        color = 'black'
    return 'color: %s' % color

weights.style.applymap(color_negative_red, subset=['diff_btc', 'diff_btc_round', 'diff'])

,name,weight,price_btc,current_value,current_value_btc,target_value_btc,diff_btc,diff_btc_round,diff
symbol,,,,,,,,,
BTC,Bitcoin,0.1,1,0.00410935,0.00410935,0.00556382,0.00145447,0.0015,0.0015
ETH,Ethereum,0.1,0.0529529,0.091908,0.0048668,0.00556382,0.00069702,0.0007,0.0132193
BCH,Bitcoin Cash,0.1,0.209343,0,0,0.00556382,0.00556382,0.0056,0.0267504
XRP,Ripple,0.100596,7.491e-05,77.846,0.00583144,0.00559696,-0.000234489,-0.0002,-2.66987
LTC,Litecoin,0.100596,0.0198713,0.21978,0.00436731,0.00559696,0.00122964,0.0012,0.0603886
MIOTA,IOTA,0.0889649,0.00032757,14.985,0.00490864,0.00494984,4.12079e-05,0,0
ADA,Cardano,0.0758675,2.995e-05,226.596,0.00678655,0.00422112,-0.00256543,-0.0026,-86.8114
DASH,Dash,0.0680886,0.0897185,0.062877,0.00564123,0.00378832,-0.00185291,-0.0019,-0.0211773
BTG,Bitcoin Gold,0.0421177,0.025781,0.14975,0.0038607,0.00234335,-0.00151735,-0.0015,-0.0581824


In [55]:
diffs = weights.loc[:, ['name', 'diff_btc_round', 'diff']]

diffs.style.applymap(color_negative_red, subset=['diff_btc_round', 'diff'])

,name,diff_btc_round,diff
symbol,,,
BTC,Bitcoin,0.0015,0.0015
ETH,Ethereum,0.0007,0.0132193
BCH,Bitcoin Cash,0.0056,0.0267504
XRP,Ripple,-0.0002,-2.66987
LTC,Litecoin,0.0012,0.0603886
MIOTA,IOTA,0,0
ADA,Cardano,-0.0026,-86.8114
DASH,Dash,-0.0019,-0.0211773
BTG,Bitcoin Gold,-0.0015,-0.0581824
